In [96]:
import pandas as pd
import numpy as np
from datetime import datetime
import itertools
import re

trainDF = pd.read_csv("train.csv")
trafoDF = pd.read_csv("trafo.csv")
submissionDF = pd.read_csv("submission.csv")
bagilNemDF = pd.read_csv("Hava Durumu/Bağıl Nem.csv")
bulutsuzlukOraniDF = pd.read_csv("Hava Durumu/Bulutluluk Oranı.csv")
radyasyonDF = pd.read_csv("Hava Durumu/Radyasyon.csv")
ruzgarHiziDF = pd.read_csv("Hava Durumu/Rüzgar Hızı.csv")
ruzgarYonuDF = pd.read_csv("Hava Durumu/Rüzgar Yönü.csv")
sicaklikDF = pd.read_csv("Hava Durumu/Sıcaklık.csv")
yagisDF = pd.read_csv("Hava Durumu/Yağış.csv")

radyasyonDF.drop("Unnamed: 17",axis=1,inplace=True) # bilinmeyen kolon ve içeriği boş olduğu için drop ediyoruz
trainDF.drop("BİLDİRİME_GÖRE",axis=1,inplace=True) # Bu kolon için tüm degerler aynı,bir katkısı olmayacak bu sebeple drop ediyoruz
trainDF.drop(["KOD_NO","KADEME"],axis=1,inplace=True) # Bu kolonların için tüm degerleri null,bir katkısı olmayacak bu sebeple drop ediyoruz
trainDF.drop("SONA_ERME_TARİHİ_VE_ZAMANI",axis=1,inplace=True) # Bitiş tarihine gerek duluymadığı için drop ediyoruz
trainDF.drop("SEBEBE_GÖRE",axis=1,inplace=True) # kolon verilerinin %98'i "Şebeke işletmecisi" türünden olduğu için bu kolonu çıkarıyoruz

def tr_capitalize(param_word):
    word_list = param_word.split(sep=" ")
    new_word = ""
    for word in word_list:
        first_letter = word[0]
        restOfWord = word[1:]

        first_letter = re.sub(r"i", "İ", first_letter)
        first_letter = re.sub(r"ı", "I", first_letter)
        first_letter = re.sub(r"ç", "Ç", first_letter)
        first_letter = re.sub(r"ş", "Ş", first_letter)
        first_letter = re.sub(r"ü", "Ü", first_letter)
        first_letter = re.sub(r"ğ", "Ğ", first_letter)

        restOfWord = re.sub(r"İ", "i", restOfWord)
        restOfWord = re.sub(r"I", "ı", restOfWord)
        restOfWord = re.sub(r"Ç", "ç", restOfWord)
        restOfWord = re.sub(r"Ş", "ş", restOfWord)
        restOfWord = re.sub(r"Ü", "ü", restOfWord)
        restOfWord = re.sub(r"Ğ", "ğ", restOfWord)

        rebuilt_word = first_letter + restOfWord
        rebuilt_word = rebuilt_word.capitalize()
        new_word = new_word + " " + rebuilt_word

    new_word = new_word.strip()
    return new_word

    
trainDF["BAŞLAMA_TARİHİ"] = pd.to_datetime(trainDF["BAŞLAMA_TARİHİ_VE_ZAMANI"], dayfirst=True).dt.date

bagilNemDF["Tarih"]=pd.to_datetime(bagilNemDF["Tarih"], dayfirst=True,errors='coerce').dt.date
radyasyonDF["Tarih"]=pd.to_datetime(radyasyonDF["Tarih"], dayfirst=True,errors='coerce').dt.date
bulutsuzlukOraniDF["Tarih"]=pd.to_datetime(bulutsuzlukOraniDF["Tarih"], dayfirst=True,errors='coerce').dt.date
ruzgarHiziDF["Tarih"]=pd.to_datetime(ruzgarHiziDF["Tarih"], dayfirst=True,errors='coerce').dt.date
ruzgarYonuDF["Tarih"]=pd.to_datetime(ruzgarYonuDF["Tarih"], dayfirst=True,errors='coerce').dt.date
sicaklikDF["Tarih"]=pd.to_datetime(sicaklikDF["Tarih"], dayfirst=True,errors='coerce').dt.date
yagisDF["Tarih"]=pd.to_datetime(yagisDF["Tarih"], dayfirst=True,errors='coerce').dt.date


minDate = trainDF["BAŞLAMA_TARİHİ"].min()

# hava durumu ile ilgili dataframelerin tarihini traindf nin en düşük tarihine göre filtreliyoruz
bagilNemDF=bagilNemDF[bagilNemDF["Tarih"] >= minDate]
bulutsuzlukOraniDF=bulutsuzlukOraniDF[bulutsuzlukOraniDF["Tarih"] >= minDate]
radyasyonDF=radyasyonDF[radyasyonDF["Tarih"] >= minDate]
ruzgarHiziDF=ruzgarHiziDF[ruzgarHiziDF["Tarih"] >= minDate]
ruzgarYonuDF=ruzgarYonuDF[ruzgarYonuDF["Tarih"] >= minDate]
sicaklikDF=sicaklikDF[sicaklikDF["Tarih"] >= minDate]
yagisDF=yagisDF[yagisDF["Tarih"] >= minDate]

trainDF['SÜREYE_GÖRE']=trainDF['SÜREYE_GÖRE'].map({"Kısa":1,"Uzun":2}).astype(int)




In [97]:
# %80 ve üzeri oranda aynı degeri taşıyan kolonları siliyoruz
#?satirSayisi = len(finalDF[finalDF["KESİNTİ_NO"].notna()])
#! trainDF["TOPLAM_KENTALTI_OG"].value_counts()/satirSayisi #%94 0
#! trainDF["KENTSEL_OG"].value_counts()/satirSayisi # %80 0
#? trainDF["KENTSEL_AG"].value_counts()/satirSayisi # +++
#! trainDF["KIRSAL_OG"].value_counts()/satirSayisi# %92 0
#! trainDF["KIRSAL_AG"].value_counts()/satirSayisi# %82.5 0
#! trainDF["TOPLAM_KENTSEL_OG"].value_counts()/satirSayisi # %80 0
#? trainDF["TOPLAM_KENTSEL_AG"].value_counts()/satirSayisi # +++
#! trainDF["TOPLAM_KENTALTI_AG"].value_counts()/satirSayisi # %83 0
#! trainDF["TOPLAM_KIRSAL_OG"].value_counts()/satirSayisi # %920
#! trainDF["TOPLAM_KIRSAL_AG"].value_counts()/satirSayisi # %82 0

trainDF.drop("TOPLAM_KENTALTI_OG",axis=1,inplace=True)
trainDF.drop("KENTSEL_OG",axis=1,inplace=True)
trainDF.drop("KIRSAL_OG",axis=1,inplace=True)
trainDF.drop("KIRSAL_AG",axis=1,inplace=True)
trainDF.drop("TOPLAM_KENTSEL_OG",axis=1,inplace=True)
trainDF.drop("TOPLAM_KENTALTI_AG",axis=1,inplace=True)
trainDF.drop("TOPLAM_KIRSAL_OG",axis=1,inplace=True)
trainDF.drop("TOPLAM_KIRSAL_AG",axis=1,inplace=True)





In [98]:
havaDurumuLokasyonlar = list(set(itertools.chain(bagilNemDF.columns,bulutsuzlukOraniDF.columns,radyasyonDF.columns,ruzgarHiziDF.columns,ruzgarYonuDF.columns,sicaklikDF.columns,yagisDF.columns)))

trainDF["İL"]=trainDF["İL"].apply(lambda x: tr_capitalize(x))
trainDF["İLÇE"]=trainDF["İLÇE"].apply(lambda x: tr_capitalize(x))
trainDF["LOKASYON"] = trainDF[["İL","İLÇE"]].apply(lambda x: x["İLÇE"] if x["İLÇE"] in havaDurumuLokasyonlar else x["İL"], axis=1)
trainDF.drop(["İL","İLÇE"],axis=1,inplace=True)

In [99]:
trainDF=pd.merge(trainDF,trafoDF,on='ŞEBEKE_UNSURU_KODU',how='left')
trainDF['target']=1
#start -> trainDF["BAŞLAMA_TARİHİ"]).min() and end -> trainDF["BAŞLAMA_TARİHİ"]).max()
dframe=pd.DataFrame(pd.date_range(start='5/1/2021', end='6/12/2022', freq='D'))
dframe.rename(columns={0:"Tarih"},inplace=True)

uniqueTrafo=pd.DataFrame()
uniqueTrafo["trafo_id"]=trainDF["trafo_id"].unique()
# uniqueTrafo=uniqueTrafo.merge(trainDF[["trafo_id","LOKASYON"]],on="trafo_id",how="inner").drop_duplicates()
# dframe=dframe.merge(uniqueTrafo[["trafo_id","LOKASYON"]], how='cross')

uniqueTrafo=uniqueTrafo.merge(trainDF["trafo_id"],on="trafo_id",how="inner").drop_duplicates()
dframe=dframe.merge(uniqueTrafo["trafo_id"], how='cross')

trainDF.rename(columns={"BAŞLAMA_TARİHİ":"Tarih"},inplace=True)
trainDF["Tarih"]=trainDF["Tarih"].astype('datetime64[ns]')


trafo_lokasyon=trainDF[["trafo_id","LOKASYON"]].drop_duplicates()
finalDF = pd.merge(dframe,trafo_lokasyon, on=['trafo_id'],how="left")

finalDF

,Tarih,trafo_id,LOKASYON
0,2021-05-01,0,Urla
1,2021-05-01,1,İzmir
2,2021-05-01,2,İzmir
3,2021-05-01,3,Bergama
4,2021-05-01,4,İzmir
...,...,...,...
26007139,2022-06-12,63762,İzmir
26007140,2022-06-12,63763,İzmir
26007141,2022-06-12,63764,İzmir
26007142,2022-06-12,63765,Akhisar


In [100]:
sebekeUnsuru_TrafoID=trainDF[["trafo_id","ŞEBEKE_UNSURU"]].drop_duplicates()
finalDF = pd.merge(finalDF,sebekeUnsuru_TrafoID, on=['trafo_id'],how="left")
trainDF.drop("ŞEBEKE_UNSURU",axis=1,inplace=True) # finalDF ye eklendiği için drop ediyoruz
finalDF

,Tarih,trafo_id,LOKASYON,ŞEBEKE_UNSURU
0,2021-05-01,0,Urla,OG Fideri
1,2021-05-01,1,İzmir,OG Fideri
2,2021-05-01,2,İzmir,TM Fideri
3,2021-05-01,3,Bergama,OG Fideri
4,2021-05-01,4,İzmir,OG Fideri
...,...,...,...,...
26007139,2022-06-12,63762,İzmir,Abone Tesisi
26007140,2022-06-12,63763,İzmir,Abone Tesisi
26007141,2022-06-12,63764,İzmir,Abone Tesisi
26007142,2022-06-12,63765,Akhisar,Abone Tesisi


In [101]:
bagilNem = bagilNemDF.melt(id_vars = 'Tarih', var_name = 'LOKASYON', value_name = 'BAĞIL_NEM').groupby(["Tarih","LOKASYON"]).mean().reset_index()
bagilNem["Tarih"]=pd.to_datetime(bagilNem["Tarih"], dayfirst=True)

bulutsuzlukOrani = bulutsuzlukOraniDF.melt(id_vars = 'Tarih', var_name = 'LOKASYON', value_name = 'BULUTSUZLUK_ORANI').groupby(["Tarih","LOKASYON"]).mean().reset_index()
bulutsuzlukOrani["Tarih"]=pd.to_datetime(bulutsuzlukOrani["Tarih"], dayfirst=True)

radyasyon = radyasyonDF.melt(id_vars = 'Tarih', var_name = 'LOKASYON', value_name = 'RADYASYON').groupby(["Tarih","LOKASYON"]).mean().reset_index()
radyasyon["Tarih"]=pd.to_datetime(radyasyon["Tarih"], dayfirst=True)

ruzgarHizi = ruzgarHiziDF.melt(id_vars = 'Tarih', var_name = 'LOKASYON', value_name = 'RUZGAR_HIZI').groupby(["Tarih","LOKASYON"]).mean().reset_index()
ruzgarHizi["Tarih"]=pd.to_datetime(ruzgarHizi["Tarih"], dayfirst=True)

ruzgarYonu = ruzgarYonuDF.melt(id_vars = 'Tarih', var_name = 'LOKASYON', value_name = 'RUZGAR_YONU').groupby(["Tarih","LOKASYON"]).mean().reset_index()
ruzgarYonu["Tarih"]=pd.to_datetime(ruzgarYonu["Tarih"], dayfirst=True)

sicaklik = sicaklikDF.melt(id_vars = 'Tarih', var_name = 'LOKASYON', value_name = 'SICAKLIK').groupby(["Tarih","LOKASYON"]).mean().reset_index()
sicaklik["Tarih"]=pd.to_datetime(sicaklik["Tarih"], dayfirst=True)

yagis = yagisDF.melt(id_vars = 'Tarih', var_name = 'LOKASYON', value_name = 'YAGIS').groupby(["Tarih","LOKASYON"]).mean().reset_index()
yagis["Tarih"]=pd.to_datetime(yagis["Tarih"], dayfirst=True)

havaDurumuDF=pd.merge(bagilNem,bulutsuzlukOrani, on=['Tarih',"LOKASYON"],how="outer")
havaDurumuDF=pd.merge(havaDurumuDF,radyasyon, on=['Tarih',"LOKASYON"],how="outer")
havaDurumuDF=pd.merge(havaDurumuDF,ruzgarHizi, on=['Tarih',"LOKASYON"],how="outer")
havaDurumuDF=pd.merge(havaDurumuDF,ruzgarYonu, on=['Tarih',"LOKASYON"],how="outer")
havaDurumuDF=pd.merge(havaDurumuDF,sicaklik, on=['Tarih',"LOKASYON"],how="outer")
havaDurumuDF=pd.merge(havaDurumuDF,yagis, on=['Tarih',"LOKASYON"],how="outer")


In [102]:
finalDF = pd.merge(finalDF,havaDurumuDF, on=['Tarih',"LOKASYON"],how="left")
finalDF = pd.merge(finalDF,trainDF, on=["Tarih",'trafo_id',"LOKASYON"],how="left")
finalDF["target"].fillna(0, inplace=True)
finalDF["target"]=finalDF["target"].astype(int)
finalDF['SÜREYE_GÖRE'].fillna(0,inplace=True)

In [105]:

#? trainDF["KENTSEL_AG"].value_counts()/satirSayisi # +++
#? trainDF["TOPLAM_KENTSEL_AG"].value_counts()/satirSayisi # +++
# quantillara bak estrem degerleri at modunu nanlara at

#****************

# len(finalDF[finalDF["BAĞIL_NEM"].isna()])/len(finalDF)*100 # 1.4706792604609773
# len(finalDF[finalDF["RADYASYON"].isna()])/len(finalDF)*100 # 1.4706792604609773
# len(finalDF[finalDF["RUZGAR_HIZI"].isna()])/len(finalDF)*100 # 1.4706792604609773
# len(finalDF[finalDF["RUZGAR_YONU"].isna()])/len(finalDF)*100 # 1.4706792604609773
# len(finalDF[finalDF["SICAKLIK"].isna()])/len(finalDF)*100 # 1.5054300684489808
# len(finalDF[finalDF["YAGIS"].isna()])/len(finalDF)*100 # 1.715753384368849


finalDF[(finalDF["SICAKLIK"].isna()) & (finalDF["target"]==1)].sort_values(by="Tarih") # 06.06.2022 sonrası hava durumu verisi olmayanlar için napacagız??








,trafo_id,BAĞIL_NEM,BULUTSUZLUK_ORANI,RADYASYON,RUZGAR_HIZI,RUZGAR_YONU,SICAKLIK,YAGIS,KESİNTİ_NO,SÜREYE_GÖRE,KESİNTİ_SÜRESİ,KENTSEL_AG,KENTALTI_OG,KENTALTI_AG,TOPLAM_KENTSEL_AG,target
Tarih,,,,,,,,,,,,,,,,
2021-05-01,33826.062083,0.006833,0.083333,300.628125,1.042333,2.523708,22.571958,0.0,1.806396e+06,0.009167,1.486465,422.272727,0.636364,50.545455,319.320429,0.004583
2021-05-02,33826.062083,0.008208,0.375000,264.201042,1.140083,2.663583,23.415625,0.0,1.806828e+06,0.005000,5.207361,358.166667,0.000000,0.166667,782.299815,0.002500
2021-05-03,33826.062083,0.007417,0.126667,297.981417,1.367875,3.076292,22.998542,0.0,1.807231e+06,0.007500,2.674321,22.555556,19.555556,134.222222,16.167531,0.003750
2021-05-04,33826.062083,0.007708,0.251667,239.125375,2.748250,2.732708,21.250000,0.0,1.807953e+06,0.004167,1.772389,277.600000,0.000000,0.000000,481.792556,0.002083
2021-05-05,33826.062083,0.008292,0.080000,270.075958,1.599792,3.046417,19.592292,0.0,1.808457e+06,0.005000,2.585417,18.166667,1.166667,95.000000,14.209861,0.002500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-06-08,33826.062083,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.312704e+06,0.005833,1.635595,28.000000,0.142857,1.571429,17.026151,0.002917
2022-06-09,33826.062083,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.313222e+06,0.008333,1.453167,208.700000,1.200000,60.000000,159.986694,0.004167
2022-06-10,33826.062083,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.313721e+06,0.004583,0.818380,204.666667,4.500000,1.000000,153.821018,0.002500


In [84]:
finalDF.corr()

,trafo_id,BAĞIL_NEM,BULUTSUZLUK_ORANI,RADYASYON,RUZGAR_HIZI,RUZGAR_YONU,SICAKLIK,YAGIS,KESİNTİ_NO,SÜREYE_GÖRE,KESİNTİ_SÜRESİ,KENTSEL_AG,KENTALTI_OG,KENTALTI_AG,TOPLAM_KENTSEL_AG,target
trafo_id,1.000000,0.023047,-0.001744,0.000997,0.030479,-0.001222,0.006032,-0.000083,-0.373086,-0.021201,0.150458,-0.127219,-0.072199,-0.056332,-0.097970,-0.021924
BAĞIL_NEM,0.023047,1.000000,-0.166006,0.411355,0.249492,0.289503,0.753557,0.031967,-0.555648,0.005065,0.052223,0.003974,-0.013391,0.025971,0.006783,0.005073
BULUTSUZLUK_ORANI,-0.001744,-0.166006,1.000000,-0.683026,0.075751,0.022846,-0.423530,0.529835,0.406685,0.004528,0.036892,-0.011241,0.014705,-0.001601,-0.008013,0.004478
RADYASYON,0.000997,0.411355,-0.683026,1.000000,-0.063180,0.194217,0.745498,-0.411104,-0.566025,-0.001660,-0.050316,0.018870,-0.012197,-0.001580,0.013127,-0.001586
RUZGAR_HIZI,0.030479,0.249492,0.075751,-0.063180,1.000000,-0.280146,0.028813,0.083722,0.123940,0.005969,0.071149,-0.019262,-0.017863,0.015335,-0.011421,0.005683
RUZGAR_YONU,-0.001222,0.289503,0.022846,0.194217,-0.280146,1.000000,0.237021,0.049108,-0.196153,0.002129,0.024730,0.011489,0.003514,0.006458,0.012618,0.002215
SICAKLIK,0.006032,0.753557,-0.423530,0.745498,0.028813,0.237021,1.000000,-0.171749,-0.669871,0.004681,0.000565,0.022743,-0.010894,0.008703,0.028542,0.004817
YAGIS,-0.000083,0.031967,0.529835,-0.411104,0.083722,0.049108,-0.171749,1.000000,0.173735,0.007989,0.047074,-0.008789,0.003253,0.004359,-0.006485,0.008034
KESİNTİ_NO,-0.373086,-0.555648,0.406685,-0.566025,0.123940,-0.196153,-0.669871,0.173735,1.000000,0.039976,-0.000729,-0.014151,0.011480,-0.009094,-0.010562,NaN
SÜREYE_GÖRE,-0.021201,0.005065,0.004528,-0.001660,0.005969,0.002129,0.004681,0.007989,0.039976,1.000000,0.152835,-0.078959,-0.083178,-0.082754,0.031613,0.996517


In [85]:

def func(values, functionName):
    
    if functionName == "log":
        return np.log(values)
    
    elif functionName == "sqrt":
        return np.sqrt(values)
    
    elif functionName == "pow2.0":
        
        return np.power(values,2.0)
        
    elif functionName == "pow3.0":
        return np.power(values,3.0)
    
    elif functionName == "pow1.5":
        return np.power(values,1.5)
    
    elif functionName == "pow1.2":
        return np.power(values,1.2)
    
    elif functionName == "pow0.7":
        return np.power(values,0.7)
    
    elif functionName == "exp":
        return np.exp(values)
    
    elif functionName == "round":
        return np.round(values)
    
    elif functionName == "round-1":
        return np.round(values,-1)
    
    elif functionName == "gtMean":
        mean =  np.mean(values)
        return mean < values
    
    elif functionName == "gtMedian":
        median = np.median(values).astype(int)
        return values > median

results = pd.DataFrame ( columns= ["self","corr","log","sqrt",
"pow2.0",
"pow3.0",
"pow1.5",
"pow1.2",
"pow0.7",
"exp",
"round",
"round-1",
"gtMean",
"getMedian"])



print(finalDF.columns)
for x in finalDF:
    if finalDF.dtypes[x] in ["float64","int64","uint8"]:
        results.loc[len(results)]= [x,
        finalDF[x].corr(finalDF["target"]),
        func(finalDF[x],"log").corr(finalDF["target"]),
        func(finalDF[x],'sqrt').corr(finalDF["target"]),
        func(finalDF[x],"pow2.0").corr(finalDF["target"]),
        func(finalDF[x],"pow3.0").corr(finalDF["target"]),
        func(finalDF[x],"pow1.5").corr(finalDF["target"]),
        func(finalDF[x],"pow1.2").corr(finalDF["target"]),
        func(finalDF[x],"pow0.7").corr(finalDF["target"]),
        func(finalDF[x],"exp").corr(finalDF["target"]),
        func(finalDF[x],"round").corr(finalDF["target"]),
        func(finalDF[x],"round-1").corr(finalDF["target"]),
        func(finalDF[x],"gtMean").corr(finalDF["target"]),
        func(finalDF[x],"gtMedian").corr(finalDF["target"])
    ]
        
#

results.to_csv("results.csv")

Index(['Tarih', 'trafo_id', 'LOKASYON', 'ŞEBEKE_UNSURU', 'BAĞIL_NEM',
       'BULUTSUZLUK_ORANI', 'RADYASYON', 'RUZGAR_HIZI', 'RUZGAR_YONU',
       'SICAKLIK', 'YAGIS', 'KESİNTİ_NO', 'ŞEBEKE_UNSURU_KODU',
       'KESİNTİ_NEDENİNE_İLİŞKİN_AÇIKLAMA', 'KAYNAĞA_GÖRE', 'SÜREYE_GÖRE',
       'BAŞLAMA_TARİHİ_VE_ZAMANI', 'KESİNTİ_SÜRESİ', 'KENTSEL_AG',
       'KENTALTI_OG', 'KENTALTI_AG', 'TOPLAM_KENTSEL_AG', 'target'],
      dtype='object')


c:\Users\eminkbc\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\core\arraylike.py:397: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
c:\Users\eminkbc\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\core\arraylike.py:397: RuntimeWarning: overflow encountered in exp
  result = getattr(ufunc, method)(*inputs, **kwargs)
c:\Users\eminkbc\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\core\arraylike.py:397: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
c:\Users\eminkbc\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\core\arraylike.py:397: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
c:\Users\eminkbc\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\core\arraylike.py:397: RuntimeWarning: invalid value encountered in sqrt
  result = getattr(u

In [86]:
new = pd.DataFrame(columns=['name','mean','median','std','kurtosis','skewness', 'upper', 'lower', '#upper', '#lower', 'cardinality']) 
for x in finalDF.columns: 
    if str(finalDF.dtypes[x]) in ['int64','float64']: 
        mea = finalDF[x].mean() 
        medi = finalDF[x].median() 
        std = round(finalDF[x].std(), 2)
        kurto = round( finalDF[x].kurtosis(), 2)
        skew = round(finalDF[x].skew(), 2)
        q1 = finalDF[x].quantile(0.25)
        q3 = finalDF[x].quantile(0.73)

        iqr = q3-q1
        upper_bound = q3 + 1.5 * iqr
        lower_bound = q1 - 1.5 * iqr
        
        nof_uppers = len(finalDF[ finalDF[ x ] > upper_bound ])
        nof_lowers = len(finalDF[ finalDF[ x ] < lower_bound ])


        cardinality = len(finalDF[x].unique()) / len(finalDF)


        new.loc[len(new)] = [x,mea,medi,std,kurto,skew, upper_bound, lower_bound, nof_uppers, nof_lowers, cardinality] 

In [95]:
# Nan degerleri son valid degere göre atadık 
# for i in finalDF[["BAĞIL_NEM","BULUTSUZLUK_ORANI","RADYASYON","RUZGAR_HIZI","RUZGAR_YONU"]]:
#     last_index=finalDF[i].last_valid_index()
#     last_index_value=finalDF.iloc[last_index][i]
#     finalDF[i].fillna(last_index_value,inplace=True)



finalDF["BAĞIL_NEM"].last_valid_index()
finalDF.iloc[finalDF["BAĞIL_NEM"].last_valid_index()]["BAĞIL_NEM"]

finalDF["BAĞIL_NEM"].groupby(["LOKASYON","Tarih"]).mean()


KeyError: 'LOKASYON'

In [137]:
#yagış ve sıcaklık ayrı incelenecek. 2021 ve 2022 de boşluklar var




finalDF["Temp_Date"]=finalDF["LOKASYON"]+str(finalDF["Tarih"].dt.year)+str(finalDF["Tarih"].dt.month)
finalDF["Temp_Date"]
  


In [ ]:
for i in finalDF[["BAĞIL_NEM","BULUTSUZLUK_ORANI","RADYASYON","RUZGAR_HIZI","RUZGAR_YONU"]]:
    last_index=finalDF[i].last_valid_index()
    last_index_value=finalDF.iloc[last_index][i]
    finalDF[i].fillna(last_index_value,inplace=True)
    finalDF["BAĞIL_NEM"]=finalDF.apply(lambda x: finalDF.groupby(["LOKASYON","Tarih"]).mean().loc[x["LOKASYON"]]["BAĞIL_NEM"])

In [89]:
# trafo_date=submissionDF["trafo_id_date"].str.split("__",expand=True)
# submissionDF["trafo_id"]  = trafo_date[0]
# submissionDF["Tarih"]  = trafo_date[1]
# submissionDF.drop("trafo_id_date",axis=1,inplace=True)